# Exploratory Analysis - Karnataka State Elections-2018
### Saurabh Gandhi


In [89]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [90]:
"""
Results scraped from ECI site.
Code for the same can be found in the parent repo.
"""
df = pd.read_csv("eci-2018-states-candidate-wise-karnataka.csv")
df['Votes'] = pd.to_numeric(df['Votes'], errors='coerce').fillna(0).astype(int)


In [91]:
df = df[df['State-code']=='S10']
df.head()

,Candidate,Party,Votes,State,Constituency,State-code,Constituency-code
0,JOLLE SHASHIKALA ANNASAHEB,Bharatiya Janata Party,78453,Karnataka,Nippani,S10,1
1,KAKASO PANDURANG PATIL,Indian National Congress,70083,Karnataka,Nippani,S10,1
2,KAMAT RAMESH ISHWAR,Bahujan Samaj Party,2326,Karnataka,Nippani,S10,1
3,ROHINI SHRIMANT DIXIT,All India Mahila Empowerment Party,1227,Karnataka,Nippani,S10,1
4,SHARAD RAMAGONDA PATIL,Independent,458,Karnataka,Nippani,S10,1


In [92]:
total_votes = df['Votes'].sum()
party_votes = df[['Votes', 'Party']].groupby('Party').sum().sort_values(by='Votes',ascending=False)
vote_shares = (100*party_votes/total_votes).reset_index()
# Top votes shares
vote_shares[:15]

,Party,Votes
0,Indian National Congress,37.782699
1,Bharatiya Janata Party,36.796155
2,Janata Dal (Secular),17.782667
3,Independent,3.945932
4,None of the Above,0.863468
5,Bahujan Samaj Party,0.371800
6,All India Mahila Empowerment Party,0.283947
7,Bharatiya Praja Paksha,0.273253
8,Karnataka Pragnyavantha Janatha Party,0.251198
9,Swaraj India,0.248126


In [93]:
party_contestants = df.groupby('Party').size().sort_values(ascending=False).reset_index()
# Number of contestants
party_contestants[:20]

,Party,0
0,Independent,1127
1,None of the Above,222
2,Bharatiya Janata Party,222
3,Indian National Congress,217
4,Janata Dal (Secular),199
5,All India Mahila Empowerment Party,174
6,Shivsena,37
7,Indian New Congress Party,32
8,Karnataka Pragnyavantha Janatha Party,30
9,Aam Aadmi Party,28


In [94]:
df_win = df.sort_values(by= ['Constituency-code', 'Votes'], ascending=[1, 0]).groupby('Constituency-code', as_index=False).first()
df_win[:2]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code
0,1,JOLLE SHASHIKALA ANNASAHEB,Bharatiya Janata Party,78453,Karnataka,Nippani,S10
1,10,SATISH. L. JARKIHOLI,Indian National Congress,55174,Karnataka,Yemkanmardi,S10


In [95]:
#apply(lambda t: t.iloc[1])
runners = df.sort_values(['Constituency-code', 'Votes'], ascending=[1, 0]).groupby('Constituency-code', as_index=False).nth(1).dropna()
df_win['Margin'] = df_win['Votes'] - runners['Votes'].values # no merge needed cos in same order
df_win['Runnerup-Party'] = runners['Party'].values
df_win[:2]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party
0,1,JOLLE SHASHIKALA ANNASAHEB,Bharatiya Janata Party,78453,Karnataka,Nippani,S10,8370,Indian National Congress
1,10,SATISH. L. JARKIHOLI,Indian National Congress,55174,Karnataka,Yemkanmardi,S10,5926,Bharatiya Janata Party


In [96]:
party_wins = df_win.groupby('Party').size().sort_values(ascending=False)
# Number of wins
party_wins

Party
Bharatiya Janata Party                   109
Indian National Congress                  71
Janata Dal  (Secular)                     39
Karnataka Pragnyavantha Janatha Party      1
Independent                                1
Bahujan Samaj Party                        1
dtype: int64

In [97]:
runner_party = df_win.groupby('Runnerup-Party').size().sort_values(ascending=False)
# Number of runner-up positions
runner_party

Runnerup-Party
Indian National Congress               120
Bharatiya Janata Party                  60
Janata Dal  (Secular)                   32
Independent                              7
Swaraj India                             1
Communist Party of India  (Marxist)      1
Bharatiya Praja Paksha                   1
dtype: int64

In [98]:
df_alliance = df.copy()
alliances1 = ['Bharatiya Janata Party']
alliances2 = ['Indian National Congress', 'Janata Dal  (Secular)']
df_alliance['alliance'] = df_alliance['Party'].map(lambda x: 'BJP' if x in alliances1 else 'INC-JDS'if x in alliances2 else x)

In [99]:
df_alliance[:4]

,Candidate,Party,Votes,State,Constituency,State-code,Constituency-code,alliance
0,JOLLE SHASHIKALA ANNASAHEB,Bharatiya Janata Party,78453,Karnataka,Nippani,S10,1,BJP
1,KAKASO PANDURANG PATIL,Indian National Congress,70083,Karnataka,Nippani,S10,1,INC-JDS
2,KAMAT RAMESH ISHWAR,Bahujan Samaj Party,2326,Karnataka,Nippani,S10,1,Bahujan Samaj Party
3,ROHINI SHRIMANT DIXIT,All India Mahila Empowerment Party,1227,Karnataka,Nippani,S10,1,All India Mahila Empowerment Party


In [100]:
alliance_local = df_alliance.groupby(['Constituency-code', 'Constituency', 'alliance'], as_index=False).sum()

In [101]:
alliance_wins = alliance_local.sort_values(by=['Constituency-code', 'Votes'], ascending=[1, 0])\
                .groupby(['Constituency-code', 'Constituency'], as_index=False).first()

In [102]:
# If alliance worked, then wins would be
alliance_wins.groupby('alliance').size().sort_values(ascending=False)

alliance
INC-JDS                                  145
BJP                                       72
Independent                                3
Karnataka Pragnyavantha Janatha Party      1
Bahujan Samaj Party                        1
dtype: int64

In [103]:
code_votes = df.groupby('Constituency-code', as_index=False).sum()
code_votes.columns = ['Constituency-code', 'Total-Votes']

In [104]:
df_winp = df_win.merge(code_votes, on='Constituency-code')
df_winp['Win%'] = (100*df_winp['Votes']/df_winp['Total-Votes']).round(2)

In [106]:
# Winners with lower vote shares
df_winp.sort_values(by=['Win%'])[:5]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%
45,14,DR. ANJALI HEMANT NIMBALKAR,Indian National Congress,24210,Karnataka,Khanapur,S10,6874,Independent,92545,26.16
30,126,D S SURESH,Bharatiya Janata Party,43562,Karnataka,Tarikere,S10,12153,Independent,143103,30.44
148,32,YASHVANTHARAYGOUDA VITTALAGOUDA PATIL,Indian National Congress,39797,Karnataka,Indi,S10,8065,Janata Dal (Secular),128164,31.05
175,57,BANDI.SIDDU,Janata Dal (Secular),36551,Karnataka,Lingsugur,S10,572,Indian National Congress,115600,31.62
121,21,SIDDU BHIMAPPA NYAMGOUD,Indian National Congress,45477,Karnataka,Jamkhandi,S10,2490,Bharatiya Janata Party,141790,32.07


In [108]:
# Winners with top vote shares
df_winp.sort_values(by=['Win%'], ascending=[0])[:5]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%
65,159,R. AKHANDA SRINIVASAMURTHY,Indian National Congress,90216,Karnataka,Pulakeshinagar,S10,74719,Janata Dal (Secular),117162,77.00
67,160,K.J GEORGE,Indian National Congress,72507,Karnataka,Sarvagnanagar,S10,47227,Bharatiya Janata Party,102444,70.78
92,184,D K SHIVAKUMAR,Indian National Congress,114420,Karnataka,Kanakapura,S10,70286,Janata Dal (Secular),168809,67.78
63,157,DR. ASHWATH NARAYAN.C.N.,Bharatiya Janata Party,64270,Karnataka,Malleshwaram,S10,40938,Indian National Congress,96259,66.77
219,97,B. SREERAMULU,Bharatiya Janata Party,46971,Karnataka,Molakalmuru,S10,30142,Independent,71375,65.81


In [109]:
df_winp['Margin%'] = (100*df_winp['Margin']/df_winp['Total-Votes']).round(2)

In [111]:
# Winners with lowest margin%
df_winp.sort_values(['Margin%'])[:5]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%,Margin%
163,46,GUTTEDAR SUBHASH RUKMAYYA,Bharatiya Janata Party,68931,Karnataka,Aland,S10,132,Indian National Congress,144498,47.70,0.09
207,86,BASAVANAGOUDA PATIL,Indian National Congress,66373,Karnataka,Hirekerur,S10,149,Bharatiya Janata Party,138306,47.99,0.11
195,75,C M NIMBANNAVAR,Bharatiya Janata Party,17886,Karnataka,Kalghatgi,S10,90,Indian National Congress,37864,47.24,0.24
177,59,PRATAPGOUDA PATIL,Indian National Congress,54286,Karnataka,Maski,S10,341,Bharatiya Janata Party,123298,44.03,0.28
84,177,B.SHIVANNA,Indian National Congress,58138,Karnataka,Anekal,S10,450,Bharatiya Janata Party,120502,48.25,0.37


In [112]:
# Winners with highest margin%
df_winp.sort_values(['Margin%'], ascending=[0])[:5]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%,Margin%
65,159,R. AKHANDA SRINIVASAMURTHY,Indian National Congress,90216,Karnataka,Pulakeshinagar,S10,74719,Janata Dal (Secular),117162,77.00,63.77
67,160,K.J GEORGE,Indian National Congress,72507,Karnataka,Sarvagnanagar,S10,47227,Bharatiya Janata Party,102444,70.78,46.10
63,157,DR. ASHWATH NARAYAN.C.N.,Bharatiya Janata Party,64270,Karnataka,Malleshwaram,S10,40938,Indian National Congress,96259,66.77,42.53
219,97,B. SREERAMULU,Bharatiya Janata Party,46971,Karnataka,Molakalmuru,S10,30142,Independent,71375,65.81,42.23
92,184,D K SHIVAKUMAR,Indian National Congress,114420,Karnataka,Kanakapura,S10,70286,Janata Dal (Secular),168809,67.78,41.64


In [114]:
# with more than 50% voteshare sorted by margin
df_winp[df_winp['Win%']>50].sort_values('Margin%')[:5]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%,Margin%
85,178,N NAGARAJU (M.T.B),Indian National Congress,74381,Karnataka,Hosakote,S10,4928,Bharatiya Janata Party,146031,50.94,3.37
199,79,SUNIL BILIYA NAIK,Bharatiya Janata Party,82738,Karnataka,Bhatkal,S10,5740,Indian National Congress,165205,50.08,3.47
20,117,H HALAPPA HARATHALU,Bharatiya Janata Party,63204,Karnataka,Sagar,S10,5514,Indian National Congress,125573,50.33,4.39
158,41,RAJKUMAR PATIL,Bharatiya Janata Party,80668,Karnataka,Sedam,S10,7200,Indian National Congress,158570,50.87,4.54
157,40,PRIYANK KHARGE,Indian National Congress,62862,Karnataka,Chittapur,S10,5946,Bharatiya Janata Party,124449,50.51,4.78


In [116]:
# Parties which won with voteshares > 50%
df_winp[df_winp['Win%']>50].groupby('Party').size().sort_values(ascending=False)

Party
Bharatiya Janata Party      52
Indian National Congress    29
Janata Dal  (Secular)       17
dtype: int64

In [118]:
# Parties which won with voteshares < 30%
df_winp[df_winp['Win%']<30].groupby('Party').size().sort_values(ascending=False)

Party
Indian National Congress    1
dtype: int64

In [121]:
# Parties which won with margin < 1%
df_winp[df_winp['Margin%']<1].groupby('Party').size().sort_values(ascending=False)

Party
Bharatiya Janata Party      5
Indian National Congress    3
Janata Dal  (Secular)       1
dtype: int64

In [122]:
df_winp['First-Two'] = 2*df_winp['Votes']-df_winp['Margin']
df_winp['First-Two%'] = (100*df_winp['First-Two']/df_winp['Total-Votes']).round(2)

In [123]:
# Places where first-two had lesser share
df_winp.sort_values('First-Two%')[:3]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%,Margin%,First-Two,First-Two%
45,14,DR. ANJALI HEMANT NIMBALKAR,Indian National Congress,24210,Karnataka,Khanapur,S10,6874,Independent,92545,26.16,7.43,41546,44.89
30,126,D S SURESH,Bharatiya Janata Party,43562,Karnataka,Tarikere,S10,12153,Independent,143103,30.44,8.49,74971,52.39
148,32,YASHVANTHARAYGOUDA VITTALAGOUDA PATIL,Indian National Congress,39797,Karnataka,Indi,S10,8065,Janata Dal (Secular),128164,31.05,6.29,71529,55.81


In [124]:
# How parties dominated in mostly bi-polar regions?
df_winp[df_winp['First-Two%']>80].groupby('Party').size().sort_values(ascending=False)

Party
Bharatiya Janata Party      75
Indian National Congress    49
Janata Dal  (Secular)       27
Independent                  1
dtype: int64

In [125]:
# How parties dominated in mostly multi-cornered regions?
df_winp[df_winp['First-Two%']<50].groupby('Party').size().sort_values(ascending=False)

Party
Indian National Congress    1
dtype: int64

In [127]:
# Places where first-two had higher share
df_winp.sort_values('First-Two%', ascending=False)[:5]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%,Margin%,First-Two,First-Two%
85,178,N NAGARAJU (M.T.B),Indian National Congress,74381,Karnataka,Hosakote,S10,4928,Bharatiya Janata Party,146031,50.94,3.37,143834,98.50
116,205,RAJESH NAIK U.,Bharatiya Janata Party,97802,Karnataka,Bantval,S10,15971,Indian National Congress,182555,53.57,8.75,179633,98.40
192,72,ABBAYYA PRASAD,Indian National Congress,39063,Karnataka,Hubli-dharwad-East,S10,6579,Bharatiya Janata Party,73162,53.39,8.99,71547,97.79
100,191,SURESH GOWDAA,Janata Dal (Secular),84873,Karnataka,Nagamangala,S10,37682,Indian National Congress,135345,62.71,27.84,132064,97.58
118,207,ANGARA.S,Bharatiya Janata Party,95205,Karnataka,Sullia,S10,26068,Indian National Congress,168412,56.53,15.48,164342,97.58


In [128]:
df = df.merge(code_votes, on='Constituency-code')

In [129]:
df['vote%'] = (100*df['Votes']/df['Total-Votes']).round(2)

# Places where party got less than 5% votes
for party in list(vote_shares['Party'][:10]):
    print party, df[(df['Party']==party) & (df['vote%']<5)].shape[0]

Indian National Congress 2
Bharatiya Janata Party 19
Janata Dal  (Secular) 67
Independent 1091
None of the Above 222
Bahujan Samaj Party 17
All India Mahila Empowerment Party 174
Bharatiya Praja Paksha 3
Karnataka Pragnyavantha Janatha Party 29
Swaraj India 10


In [130]:
alliance_local = alliance_local.merge(code_votes, on='Constituency-code')

In [131]:
alliance_local['vote%'] = (100*alliance_local['Votes']/alliance_local['Total-Votes']).round(2)

In [132]:
# Even under alliance, INC-JDS would have got less than 10% votes in
alliance_local[(alliance_local['alliance']=='INC-JDS') & (alliance_local['vote%']<10)]

,Constituency-code,Constituency,alliance,Votes,Total-Votes,vote%
1710,97,Molakalmuru,INC-JDS,3184,71375,4.46


In [134]:
# high nota regions
notapa=df[(df['Party']=='None of the Above')][['Constituency-code','Constituency','Votes','vote%']]\
    .merge(df_winp[['Candidate','Party','Runnerup-Party','Constituency-code','Votes','Margin','Total-Votes','Win%','Margin%','First-Two','First-Two%']],
           on='Constituency-code', suffixes=('_nota', '_win'))

In [136]:
# Highe NOTA regions
notapa.sort_values('vote%', ascending=False)[:5]

,Constituency-code,Constituency,Votes_nota,vote%,Candidate,Party,Runnerup-Party,Votes_win,Margin,Total-Votes,Win%,Margin%,First-Two,First-Two%
170,172,B.T.M.Layout,1255,1.79,RAMALINGA REDDY,Indian National Congress,Bharatiya Janata Party,38274,16116,70069,54.62,23.00,60432,86.25
155,157,Malleshwaram,1702,1.77,DR. ASHWATH NARAYAN.C.N.,Bharatiya Janata Party,Indian National Congress,64270,40938,96259,66.77,42.53,87602,91.01
169,171,Padmanaba Nagar,1585,1.64,R ASHOKA,Bharatiya Janata Party,Janata Dal (Secular),51454,22740,96582,53.27,23.54,80168,83.01
159,161,C.V. Raman Nagar,444,1.64,S. RAGHU,Bharatiya Janata Party,Indian National Congress,12783,4397,27136,47.11,16.20,21169,78.01
162,164,Gandhi Nagar,1223,1.63,DINESH GUNDU RAO,Indian National Congress,Janata Dal (Secular),29970,6143,74864,40.03,8.21,53797,71.86


In [138]:
# Big NOTA affect at 20 places, places where candidate won by margin < NOTA votes
notapa[notapa['Votes_nota']>notapa['Margin']].sort_values('Margin%')

,Constituency-code,Constituency,Votes_nota,vote%,Candidate,Party,Runnerup-Party,Votes_win,Margin,Total-Votes,Win%,Margin%,First-Two,First-Two%
45,46,Aland,1256,0.87,GUTTEDAR SUBHASH RUKMAYYA,Bharatiya Janata Party,Indian National Congress,68931,132,144498,47.70,0.09,137730,95.32
85,86,Hirekerur,845,0.61,BASAVANAGOUDA PATIL,Indian National Congress,Bharatiya Janata Party,66373,149,138306,47.99,0.11,132597,95.87
74,75,Kalghatgi,350,0.92,C M NIMBANNAVAR,Bharatiya Janata Party,Indian National Congress,17886,90,37864,47.24,0.24,35682,94.24
58,59,Maski,1851,1.50,PRATAPGOUDA PATIL,Indian National Congress,Bharatiya Janata Party,54286,341,123298,44.03,0.28,108231,87.78
174,177,Anekal,1194,0.99,B.SHIVANNA,Indian National Congress,Bharatiya Janata Party,58138,450,120502,48.25,0.37,115826,96.12
218,221,Hanur,845,0.78,DR. PREETHAN NAGAPPA,Bharatiya Janata Party,Indian National Congress,37474,418,107693,34.80,0.39,74530,69.21
56,57,Lingsugur,1511,1.31,BANDI.SIDDU,Janata Dal (Secular),Indian National Congress,36551,572,115600,31.62,0.49,72530,62.74
151,152,Byatarayanapura,1757,0.86,A RAVI,Bharatiya Janata Party,Indian National Congress,91071,1296,205110,44.40,0.63,180846,88.17
196,199,Sakleshpur,1297,0.99,SOMASHEKAR JAYARAJ,Bharatiya Janata Party,Janata Dal (Secular),48313,943,130723,36.96,0.72,95683,73.20
65,66,Gadag,1887,1.24,HANAMANTAGOUDA KRISHNAGOUDA PATIL,Indian National Congress,Bharatiya Janata Party,73553,1690,151960,48.40,1.11,145416,95.69


In [139]:
# At these NOTA places, Shivsena won by a NOTA-whisker
notapa[notapa['Votes_nota']>notapa['Margin']].groupby('Party').size().sort_values(ascending=False)

Party
Indian National Congress    6
Bharatiya Janata Party      5
Janata Dal  (Secular)       1
dtype: int64

In [140]:
PARTIES = alliances1 + alliances2 + ['None of the Above', 'All India Majlis-E-Ittehadul Muslimeen', 'Maharashtra Navnirman sena']
df_pivot_p = df[df['Party'].isin(PARTIES)].pivot(index='Constituency-code', columns='Party', values='Votes').reset_index()
df_pivot_p[:2]

Party,Constituency-code,Bharatiya Janata Party,Indian National Congress,Janata Dal (Secular),None of the Above
0,1,78453.0,70083.0,NaN,1281.0
1,10,49248.0,55174.0,1491.0,1303.0


In [141]:
df_pivot_p = df_winp.merge(df_pivot_p, on='Constituency-code')

In [142]:
df_pivot_p[:2]

,Constituency-code,Candidate,Party,Votes,State,Constituency,State-code,Margin,Runnerup-Party,Total-Votes,Win%,Margin%,First-Two,First-Two%,Bharatiya Janata Party,Indian National Congress,Janata Dal (Secular),None of the Above
0,1,JOLLE SHASHIKALA ANNASAHEB,Bharatiya Janata Party,78453,Karnataka,Nippani,S10,8370,Indian National Congress,154681,50.72,5.41,148536,96.03,78453.0,70083.0,NaN,1281.0
1,10,SATISH. L. JARKIHOLI,Indian National Congress,55174,Karnataka,Yemkanmardi,S10,5926,Bharatiya Janata Party,108677,50.77,5.45,104422,96.08,49248.0,55174.0,1491.0,1303.0


In [144]:
top_independents = df[df['Party']=='Independent'].groupby('Constituency-code', as_index=False).first()

In [145]:
# At places where top independent got < 5% vote share
top_independents[top_independents['vote%']<5].shape[0]

184

In [146]:
# At places where top independent got > 20% vote share
top_independents[top_independents['vote%']>20].shape[0]

9

---
#### Find anything interesting? Fork, add and submit =)